This notebook is for testing the application. Read the comments. You can play around with it. 

Make sure to have your api key and your model name configured in `services/config/personal_constants.py`

In [2]:
# importing the class from a different file
from services.querier.querier import DbQuerier

# instantiating the class
test_querier = DbQuerier(sample_db_loc='data/PostNL_SQLite.sqlite')

# do some configurations
test_querier.remove_all_training_data() # if there was any, now there is not
test_querier.train_model(train_on_documentation=True, 
                         train_on_ddl=True, 
                         train_on_sql=False, 
                         train_on_question_sql_pairs=True)

INFO:root:Training data removed
Adding documentation....
Adding documentation....
Adding documentation....
INFO:root:Trained model on documentation files
Adding ddl: CREATE TABLE collo_packages (
	id INTEGER NOT NULL, 
	account_id_hashed VARCHAR, 
	dn_barcode VARCHAR, 
	da_datum_voormelding DATE, 
	da_datum_acceptatie DATE, 
	da_tijd_acceptatie TIME, 
	sa_dag_sortering1 DATE, 
	sa_datum_sortering1 DATE, 
	sa_tijd_sortering1 TIME, 
	sa_datum_distributiecollectie DATE, 
	sa_tijd_distributiecollectie TIME, 
	da_datum_herroutering_voor_up1 DATE, 
	da_tijd_herroutering_voor_up1 TIME, 
	da_datum_eindstatus DATE, 
	da_tijd_eindstatus TIME, 
	ma_gewicht INTEGER, 
	ma_breedte INTEGER, 
	ma_lengte INTEGER, 
	ma_hoogte INTEGER, 
	ma_volume INTEGER, 
	"PC4_gea" INTEGER, 
	da_landcode_gea VARCHAR, 
	da_resultaatgroepcode VARCHAR, 
	da_resultaatcode VARCHAR, 
	da_type_adres_gea VARCHAR, 
	da_waarnemingsequence VARCHAR, 
	PRIMARY KEY (id), 
	FOREIGN KEY(account_id_hashed) REFERENCES delivery_preferen

In [3]:
# showing the training data that is in the model
test_querier.training_data

,id,training_data_type,question,content
0,73575-ddl,ddl,None,CREATE TABLE delivery_preference (\n\taccount_...
1,223276-sql,sql,Hoeveel pakketten werden ontvangen door klante...,SELECT COUNT(DISTINCT cp.id) AS total_packages...
2,73574-ddl,ddl,None,CREATE TABLE delivery_facts (\n\tid INTEGER NO...
3,387140-doc,documentation,None,De 'delivery_facts' tabel bevat geaggregeerde ...
4,387141-doc,documentation,None,De 'collo_packages' tabel bevat informatie ove...
5,223273-sql,sql,Hoeveel pakketten zijn in totaal afgeleverd aa...,SELECT SUM(number_of_parcels) FROM delivery_fa...
6,73573-ddl,ddl,None,CREATE TABLE collo_packages (\n\tid INTEGER NO...
7,223266-sql,sql,Welke maand had het hoogste aantal eerste bezo...,"SELECT month_id, SUM(parcels_home_1st) AS tota..."
8,223271-sql,sql,Hoeveel klanten hebben precies 3 pakketten in ...,SELECT COUNT(*) FROM delivery_facts WHERE numb...
9,223265-sql,sql,Hoeveel klanten hebben meer dan 5 pakketten on...,SELECT COUNT(DISTINCT account_id_hashed) FROM ...


In [4]:
# some code to play around with
question = """
Welke pakketen liggen klaar om opgehaald te worden door de klant?
"""

sql_code = test_querier.generate_sql(question)
print(sql_code)

SELECT id, dn_barcode FROM collo_packages WHERE da_resultaatgroepcode = '04-Retour' AND da_resultaatcode = '04-Uitgereikt via afhaalkantoor';


In [6]:
sample_data = test_querier.generate_sample_data(sql_query=sql_code)
sample_data

,id,dn_barcode


In [18]:
sample_table = test_querier.generate_sample_data(sql_query="SELECT month_id, SUM(parcels_home_1st) AS total_first_attempts FROM delivery_facts WHERE month_id LIKE '2023%' GROUP BY month_id ORDER BY total_first_attempts DESC LIMIT 2")

In [19]:
sample_table

,month_id,total_first_attempts
0,202311,218316
1,202312,202361


In [33]:
for row in sample_table.iterrows():
    print(row[1].keys())

Index(['month_id', 'total_first_attempts'], dtype='object')
Index(['month_id', 'total_first_attempts'], dtype='object')


# Making the documentation JSON

In [48]:
dirname = "/Users/mehdigreefhorst/Desktop/data-consultancy-post/services/querier/training-data/documentation"
import os
documenation_files = []
for documenation_file in os.listdir(dirname):
    with open(os.path.join(dirname, documenation_file)) as f:
        documenation_files.append(f.read().split("\n\n"))

documenation_dict = {}
for documenation in documenation_files:
    documenation_dict[documenation[0]] = {}
    for line in documenation:
        if ":" in line:
            key_value = line.split(":")
            documenation_dict[documenation[0]][key_value[0]]=key_value[1]
        else:
            documenation_dict[documenation[0]]["table"] = line

documenation_dict

{"De 'delivery_facts' tabel bevat geaggregeerde informatie over het aantal pakketen dat elke klant ooit heeft ontvangen.": {'table': "De 'delivery_facts' tabel bevat geaggregeerde informatie over het aantal pakketen dat elke klant ooit heeft ontvangen.",
  'account_id_hashed': " Deze kolom bevat een unieke identificatiecode voor elk klantaccount. De waarden in deze kolom zijn niet uniek binnen deze tabel, aangezien er meerdere records kunnen zijn voor dezelfde klant. Deze kolom dient als vreemde sleutel die verwijst naar de primaire sleutel in de 'delivery_preference' tabel.",
  'month_id': " Dit veld bevat de maand en het jaar waarin één of meerdere pakketten aan de klant zijn geleverd, opgeslagen in het formaat 'yyyymm'. Bijvoorbeeld, '202312' staat voor december 2023.",
  'number_of_parcels': ' Deze kolom geeft het totale aantal pakketten aan dat aan de klant is geleverd. Het omvat alle pakketten ongeacht de leveringsmethode of het aantal pogingen.',
  'parcels_home_1st': ' Dit veld

In [51]:
import json
with open("services/querier/training-data/documentation/documentation_table.json", "w") as f:
    json.dump(documenation_dict, f, indent=4)

In [2]:
from services.querier.querier import DbQuerier
test_querier = DbQuerier(sample_db_loc='data/PostNL_SQLite.sqlite')
test_querier.get_descriptions_for_given_columns(["month_id", "parcels_home_1st"])

{'month_id': " Dit veld bevat de maand en het jaar waarin één of meerdere pakketten aan de klant zijn geleverd, opgeslagen in het formaat 'yyyymm'. Bijvoorbeeld, '202312' staat voor december 2023.",
 'Retrieved from table: delivery_facts': "De 'delivery_facts' tabel bevat geaggregeerde informatie over het aantal pakketen dat elke klant ooit heeft ontvangen.",
 'parcels_home_1st': ' Dit veld geeft het aantal pakketten aan dat succesvol bij de eerste bezorgpoging aan huis is afgeleverd. Het sluit pakketten uit die zijn afgehaald bij afhaalpunten of die bij latere bezorgpogingen zijn afgeleverd.'}